In [31]:
import pandas as pd
import numpy as np
import keras
from tqdm import tqdm

In [32]:
pd.set_option('display.max_columns', 40)

In [33]:
# トレーニング用の入力データの選択
#X_columns = ['locality', 'age', 'rank', 'leg', 'racing piont', \
#             'S', 'B', 'Nige', 'Maki', 'Sashi', 'Ma', \
#             '1st', '2nd', '3rd', 'Chakugai', 'win', '2ren', '3ren']
X_columns = ['1st', '2nd', '3rd', 'Chakugai', 'win', '2ren', '3ren']


# Define Model

In [34]:
from keras.models import load_model

# 関数、クラス　定義

In [35]:
def get_predict_data(df_predict):
    X = []
    race_info = []
        
    # 各レース毎に
    grouped = df_predict.groupby(['date', 'place', 'race_num'])
    for race_name, group in tqdm(grouped):
        #print(race_name)
        racer_count = group.shape[0]
        # もし、９輪ではないレースは、トレーニングの対象から外す（モデルを固めるため）
        if racer_count != 9:
            continue
        X.append(group[X_columns].values)
        race_info.append(race_name)

    X = np.array(X, dtype='float')
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

    return X, race_info

# 予測

### クローラーで取得したデータを読み込む

In [36]:
filename = 'model/DNN_model.h5'
model = load_model(filename)

In [37]:
df_predict = pd.read_csv('df_predict.csv')

In [38]:
print("Generating Predict Data")
X_pred, race_info = get_predict_data(df_predict)

Generating Predict Data


100%|█████████████████████████████████████████| 51/51 [00:00<00:00, 772.72it/s]


### 学習したモデルを元にレース結果を予測

In [39]:
print("Predicting ...")
Y_pred = model.predict(X_pred)

for index in range(len(Y_pred)):
    print('レース', race_info[index])
    print('予想順位（車番左から１ -> 9位）', np.argsort(-Y_pred[index]) + 1)
    print('勝率予測', Y_pred[index])

Predicting ...
レース (20181004, 'iwakitaira', 1)
予想順位（車番左から１ -> 9位） [9 1 2 3 7 5 4 8 6]
勝率予測 [0.13146485 0.12257328 0.12127708 0.09929096 0.11279784 0.07165696
 0.11555747 0.08382829 0.1415532 ]
レース (20181004, 'iwakitaira', 2)
予想順位（車番左から１ -> 9位） [3 1 5 2 7 9 4 8 6]
勝率予測 [0.13333538 0.11846112 0.13888098 0.09173982 0.12765794 0.07638612
 0.11735394 0.08643808 0.10974663]
レース (20181004, 'iwakitaira', 3)
予想順位（車番左から１ -> 9位） [7 1 2 9 3 5 4 8 6]
勝率予測 [0.13518658 0.12476886 0.12018583 0.08932323 0.11473625 0.06547166
 0.14942296 0.07650087 0.12440374]
レース (20181004, 'iwakitaira', 4)
予想順位（車番左から１ -> 9位） [7 1 2 9 5 3 4 8 6]
勝率予測 [0.12821414 0.1204653  0.10496501 0.08517926 0.11767896 0.06480891
 0.18110876 0.07910927 0.11847037]
レース (20181004, 'iwakitaira', 5)
予想順位（車番左から１ -> 9位） [1 9 5 2 3 7 4 8 6]
勝率予測 [0.17394124 0.1173142  0.11146932 0.09365657 0.11860861 0.07083885
 0.11113715 0.07776182 0.12527226]
レース (20181004, 'iwakitaira', 6)
予想順位（車番左から１ -> 9位） [9 3 1 2 7 5 4 8 6]
勝率予測 [0.12269781 0.12076